In [2]:
import torch
import torchvision.transforms as T
from torchcam.methods import GradCAM
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import cv2
import numpy as np 
from PIL import Image

/home/pnaray2s/miniconda3/envs/sss/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
feature_maps = {}

def save_feature(name):
    def hook(module, input, output):
        feature_maps[name] = output.detach().cpu()
    return hook

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 7
model = smp.Unet(encoder_name="resnet34", encoder_weights=None, in_channels=3, classes=num_classes)

# model, model_type = load_model(checkpoint_path, num_classes)

checkpoint = torch.load("/home/pnaray2s/workspace/Learnings/Xai/sss/my_checkpoints/H1_Experiments/data_processed_citypark_25m_H1_Experiments_citypark_synthetic_25m_pretrained_unet_unet_miou0.5013_epoch60_best.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint)

layer_name = "bottleneck"
handle = model.decoder.center.register_forward_hook(save_feature(layer_name))